In [117]:
from elasticsearch import Elasticsearch
import json

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        },
        "aggs": {
            "twitter_accounts_in_range": {
                "cardinality" : {
                    "field" : "uid"
                }
            }
        }
    }
)

print (json.dumps(response, indent = 2))
# print (response["aggregations"]["twitter_accounts_in_range"]["value"])

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.022s]


{
  "took": 10,
  "_shards": {
    "successful": 5,
    "total": 5,
    "failed": 0
  },
  "timed_out": false,
  "hits": {
    "max_score": 0.0,
    "total": 4945,
    "hits": []
  },
  "aggregations": {
    "twitter_accounts_in_range": {
      "value": 106
    }
  }
}


In [116]:
from elasticsearch import Elasticsearch

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        }
    }
)

# print (json.dumps(response, indent = 2))
print (json.dumps(response["hits"]["total"]))

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.009s]


4945


In [109]:
from elasticsearch import Elasticsearch
from bokeh.charts import Bar, reset_output
# from bokeh.io import output_notebook
import json

client = Elasticsearch('localhost:9200')

response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        },
        "aggs": {
            "tweets_by_month": {
                "date_histogram": {
                    "field": "created_at",
                    "interval": "month",
                    "format": "yyyy-MM",
                    "min_doc_count" : 0
                },
                "aggs" : {
                    "users_per_month" : {
                        "cardinality" : {
                            "field" : "uid"
                        }
                    }
                }
            }
        }
    }
)


data = dict(month=[], users_per_month=[])

for d in response["aggregations"]["tweets_by_month"]["buckets"]:
    data["month"].append(d["key_as_string"])
    data["users_per_month"].append(d["users_per_month"]["value"])
# print (data)
# bar = Bar(data, label="month", values="users_per_month")
reset_output()
output_notebook()
show(Bar(data, label="month", values="users_per_month", legend=False, color="black", plot_width=800), notebook_handle=True)
# print (json.dumps(response, indent = 2))
# print (response["aggregations"]["users"]["value"])

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.014s]


Loading BokehJS ...

In [110]:
from elasticsearch import Elasticsearch
# from bokeh.io import output_notebook
import json

client = Elasticsearch('localhost:9200')

# search_type=count means return 
response = client.search(
    index = "twitter2",
    body = {
        "size" : 0,
        "_source": {
            "exclude": [ "user.*", "retweeted_status.*" ]
        },
        "query": {
            "bool": {
                "must": [
                    {
                        "match": {
                            "text": {
                                "type": "boolean",
                                "query": "Reversing Vulnerability CVE exploitation Spoofing password DDoS SSL CryptoLocker Encrypt",
                                "operator": "or"
                            }
                        }
                    },
                    {
                        "range": {
                            "created_at": {
                                "gte": "Sun Jan 01 00:00:00 +0000 2012",
                                "lt": "Thu Jan 01 00:00:00 +0000 2015"
                            }
                        }
                    }
                ]
            }
        },
        "aggs": {
            "tweets_by_month": {
                "date_histogram": {
                    "field": "created_at",
                    "interval": "month",
                    "format": "yyyy-MM",
                    "min_doc_count" : 0
                }
            }
        }
    }
)

data = dict(month=[], tweets_per_month=[])
for d in response["aggregations"]["tweets_by_month"]["buckets"]:
    data["month"].append(d["key_as_string"])
    data["tweets_per_month"].append(d["doc_count"])

reset_output()
output_notebook()
show(Bar(data, label="month", values="tweets_per_month", legend=False, color="black", plot_width=800),
     notebook_handle=True)
# print ( json.dumps(response, indent = 2) )
# print (response["aggregations"]["users"]["value"])

INFO:elasticsearch:GET http://localhost:9200/twitter2/_search [status:200 request:0.009s]


Loading BokehJS ...